In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
from typing import List, Tuple

# Load API key from environment or .env file
load_dotenv()  # reads .env if present
API_KEY = os.getenv("GOOGLE_API_KEY")
if not API_KEY:
    raise ValueError("Chưa có GOOGLE_API_KEY trong biến môi trường hoặc file .env")

genai.configure(api_key=API_KEY)
MODEL_NAME = "gemini-1.5-flash" #model

model = genai.GenerativeModel(MODEL_NAME)
print("Configured model:", MODEL_NAME)

Configured model: gemini-1.5-flash


d:\DA_CSI01\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Multi-turn helper: giữ tối đa `max_turns` lượt (mỗi lượt = user + assistant)
from collections import deque

class MultiTurnChat:
    def __init__(self, system_prompt=None):
        self.history = []
        if system_prompt:
            self.history.append({
                "role": "system",
                "parts": [{"text": system_prompt}]
            })

    def add_user(self, text):
        self.history.append({
            "role": "user",
            "parts": [{"text": text}]
        })

    def add_assistant(self, text):
        self.history.append({
            "role": "model",
            "parts": [{"text": text}]
        })

    def as_generative_input(self):
        return self.history

# Demo
chat = MultiTurnChat(max_turns=2)
chat.add_user('Xin chào, bạn là ai?')
chat.add_assistant('Mình là FruitBot, mình giúp bạn về các loại trái cây.')
chat.add_user('Chuối chín thì ăn như nào ngon?')
print(chat.get_history_messages())


[('user', 'Xin chào, bạn là ai?'), ('assistant', 'Mình là FruitBot, mình giúp bạn về các loại trái cây.'), ('user', 'Chuối chín thì ăn như nào ngon?')]


In [4]:
# Tạo mô hình Gemini với system prompt
def ask_fruitbot(chat: MultiTurnChat, user_text: str, model_obj=model, temperature: float = 0.2, max_output_tokens: int = 256):
    # Add user message to history
    chat.add_user(user_text)
    # Build messages array
    messages = chat.as_generative_input()  # giả sử trả về list các dict hoặc string

    # Call genai to generate a response
    resp = model_obj.generate_content(
        contents=messages,
        generation_config={
            "temperature": temperature,
            "max_output_tokens": max_output_tokens
        }
    )
    bot_text = resp.text if hasattr(resp, 'text') else str(resp)
    # Save assistant's reply into history
    chat.add_assistant(bot_text)
    return bot_text

# Quick interactive demo in notebook
reply = ask_fruitbot(chat, "Món trái cây nào nhiều vitamin C nhất?")
print('FruitBOT:', reply)


KeyError: "Unable to determine the intended type of the `dict`. For `Content`, a 'parts' key is expected. For `Part`, either an 'inline_data' or a 'text' key is expected. For `Blob`, both 'mime_type' and 'data' keys are expected. However, the provided dictionary has the following keys: ['role', 'content']"

In [ ]:
# Bạn có thể lưu lịch sử chat vào file JSON nếu muốn persist giữa các lần chạy
import json

def save_history(chat: MultiTurnChat, path: str = 'chat_history.json'):
    data = chat.get_history_messages()
    with open(path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

def load_history(path: str = 'chat_history.json') -> MultiTurnChat:
    c = MultiTurnChat(max_turns=2)
    try:
        with open(path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        for role, text in data:
            if role == 'user':
                c.add_user(text)
            else:
                c.add_assistant(text)
    except FileNotFoundError:
        pass
    return c


PhoBot sẵn sàng hỗ trợ. Gõ 'exit' để thoát.

PhoBot: Chào bạn! Có gì mình giúp được không?

PhoBot: Mình là FruitBot, trợ lý AI giúp bạn tìm hiểu về trái cây.

PhoBot: Trái cây là phần quả của thực vật, thường có vị ngọt và chứa nhiều chất dinh dưỡng.

PhoBot: Có gì mình có thể giúp bạn?

PhoBot: Bạn có câu hỏi nào về trái cây không?

